<style>
pre > code {
    background-color: #3A3960 !important;
    padding: 10px;
    display: block;
    border-radius: 5px;
    border: 1px solid #ccc;
    overflow-x: auto;
}
</style>

# Python requests-Modul: Web Scraping

Web Scraping ist eine Technik, mit der man automatisiert Daten von Webseiten extrahieren kann. Anstatt manuell Inhalte von einer Webseite zu kopieren, kann ein Web Scraper dies automatisch tun. Dabei ruft ein Programm die HTML-Seite einer Webseite auf, analysiert die Struktur und extrahiert relevante Informationen.

**Wie funktioniert Web Scraping?**

Der Prozess des Web Scraping läuft in folgenden Schritten ab:
1. Anfrage senden – Das Skript sendet eine HTTP-Request (z. B. mit requests.get()) an eine Webseite.
2. Antwort erhalten – Die Webseite antwortet mit HTML-Code, JavaScript oder JSON-Daten.
3. Daten extrahieren – Der HTML-Code wird mit einer Bibliothek wie BeautifulSoup analysiert.
4. Daten speichern – Die extrahierten Daten werden z. B. in einer Datenbank oder CSV-Datei gespeichert.

**Wo wird Web Scraping genutzt?**

1. Preisvergleich: Automatische Preisüberwachung für E-Commerce-Websites.
2. Datenanalyse: Analyse von Nachrichten, Social Media oder Blogs.
3. SEO & Marketing: Analyse von Konkurrenz-Websites und Keywords.
4. Automatisierte Recherchen: Sammeln von Unternehmensdaten oder Kontaktinformationen.
5. Börsen- und Finanzdaten: Automatische Erfassung von Aktienkursen oder Kryptopreisen.

## Einführungsbeispiel

Wir werden uns am Anfang auf die folgende Website konzentrieren: https://www.scrapethissite.com/pages/forms/
<br>
<br>
Als erstes müssen folgende Module installiert werden:
```
pip3 install beautifulsoup4
pip3 install requests
```

### Schritt 1: HTML der Webseite abrufen

Nun können wir den Inhalt der Website uns ausgeben lassen:

```python
from bs4 import BeautifulSoup
import requests

URL = "https://www.scrapethissite.com/pages/forms/"

response  = requests.get(URL)
content = response.text
print(f"Response: {content}")
```

Durch "response.text" wird der HTML-Code der Webseite als String zurück gegeben. Dieser HTML-Code enthält die gesamte Struktur der Seite (z. B. `<html>`, `<head>`, `<body>`, `<div>`, etc.). Der HTML-Code kann sehr lang sein, da er die gesamte Struktur der Webseite enthält. 

### Schritt 2: HTML mit BeautifulSoup parsen

Was kann man nun mit dieser HTML-Antwort machen? Die HTML-Daten sind momentan nur als reiner Text verfügbar. Um sie zu strukturieren und gezielt auszulesen, nutzt man BeautifulSoup:

```python
from bs4 import BeautifulSoup
import requests

URL = "https://www.scrapethissite.com/pages/forms/"

response  = requests.get(URL)
content = response.text

soup = BeautifulSoup(content, "html.parser")

# Mit "prettify()" wird der Code schöner ausgegeben
print(f"Response: {soup.prettify()}")
```

Der Befehl `BeautifulSoup(content, "html.parser")` analysiert den HTML-Code und stellt ihn in einer baumartigen Struktur dar, die einfach durchsucht und bearbeitet werden kann.<br>
"html.parser": Gibt an, dass der HTML-Code mit Pythons eingebautem HTML-Parser analysiert werden soll. Es gibt jedoch noch viele andere Parses die genutzt werden können.

### Schritt 3: Bestimmte Elemente auslesen

Nachdem der HTML-Code geparsed wurde, können wir sehr einfach jedes einzelne Element aus der HTML-Struktur ansprechen und auslesen. Wir wollen alle NHL-Teams als Liste haben:

```python
from bs4 import BeautifulSoup
import requests

URL = "https://www.scrapethissite.com/pages/forms/"

response  = requests.get(URL)
content = response.text

soup = BeautifulSoup(content, "html.parser")

teams = soup.find_all("tr", class_="team")
print(teams)
```

Die Methode `find_all()` sucht in der gesamten HTML-Struktur nach allen passenden Elementen:
- Der erste Parameter ("tr") gibt an, dass alle `<tr>`-Elemente (Tabellenzeilen) durchsucht werden sollen.
- Der zweite Parameter (class_="team") schränkt die Suche weiter ein, sodass nur `<tr>`-Elemente mit der CSS-Klasse "team" zurückgegeben werden.
- Gibt eine Liste von Tag-Objekten zurück, also alle gefundenen tr-Elemente mit der Klasse "team".
- Mit einem Indexer kann man auf jedes Element der Liste dann zugreifen z.B. `print(teams[0])`

Jetzt möchten wir in jedem Tag-Objekt nach `<tr>`-Elementen suchen, mit der Klasse "team":

```python
from bs4 import BeautifulSoup
import requests

URL = "https://www.scrapethissite.com/pages/forms/"

response  = requests.get(URL)
content = response.text

soup = BeautifulSoup(content, "html.parser")

teams = soup.find_all("tr", class_="team")

for team in teams:
    team_name = team.find("td", class_="name").text.strip()
    print(f"Team: {team_name}")
```

Die Methode `find()` wird verwendet, um das erste gefundene HTML-Element zu suchen, das bestimmten Kriterien entspricht: `element = soup.find("tag_name", attributes)`
- tag_name: Der HTML-Tag, den du suchst (z. B. "td", "div", "span").
- attributes: Ein optionales Dictionary oder benannte Parameter zur Filterung (z. B. class_="name").

Das Attribut "text" und die Methode "strip()":
- .text extrahiert den Textinhalt aus dem `<td>`-Element.
- .strip() entfernt überflüssige Leerzeichen am Anfang und Ende.

Unterschied zwischen find() und find_all():
- find(): Gibt das erste gefundene Element zurück (oder None, falls nicht vorhanden)
- find_all(): Gibt eine Liste aller gefundenen Elemente zurück

### Schritt 4: Mehrere Datenfelder extrahieren

Neben den Teamnamen möchten wir weitere Informationen sammeln:
- Jahr (year)
- Siege (wins)
- Niederlagen (losses)
- Tore geschossen (GF) (gf)
- Tore kassiert (GA) (ga)
- Tor-Differenz (diff)

Um mit den Daten besser umzugehen verwenden wir pandas:

```
pip3 install pandas
```

Jetzt können wir mit sehr einfachen Methoden viele Daten extrahieren und in ein DataFrame speichern:

```python
from bs4 import BeautifulSoup
import requests
import pandas as pd

URL = "https://www.scrapethissite.com/pages/forms/"

response  = requests.get(URL)
content = response.text

soup = BeautifulSoup(content, "html.parser")

teams = soup.find_all("tr", class_="team")
data = []

for team in teams:
    team_name = team.find("td", class_="name").text.strip()
    year = team.find("td", class_="year").text.strip()
    wins = team.find("td", class_="wins").text.strip()
    losses = team.find("td", class_="losses").text.strip()
    goals_for = team.find("td", class_="gf").text.strip()
    goals_against = team.find("td", class_="ga").text.strip()
    diff = team.find("td", class_="diff").text.strip()

    data.append([team_name, year, wins, losses, goals_for, goals_against, diff])

df = pd.DataFrame(data, columns=["Team", "Jahr", "Siege", "Niederlagen", "GF", "GA", "+/-"])
print(df)
```

### Schritt 5: Speichern als CSV

```python
df.to_csv("nhl_teams.csv", index=False)
```

### Schritt 6: Umgang mit Pagination

Viele Webseiten zeigen nur eine begrenzte Anzahl von Daten pro Seite und nutzen Pagination (Seiten-Navigation), um mehr Inhalte bereitzustellen. In unserem Fall gibt es mehrere Seiten mit NHL-Teams, die über die `?page_num=`-Parameter in der URL aufgerufen werden. Die Links zur nächsten Seite folgen diesem Muster:

```
https://www.scrapethissite.com/pages/forms/?page_num=1
https://www.scrapethissite.com/pages/forms/?page_num=2
https://www.scrapethissite.com/pages/forms/?page_num=3
...
```

Die Lösung: Man muss eine Schleife erstellen welche jede Seite durchläuft.

```python
from bs4 import BeautifulSoup
import requests
import pandas as pd

BASE_URL = "https://www.scrapethissite.com/pages/forms/"
PAGE_PARAM = "?page_num="

all_teams_data = []

for page_num in range(1, 25):
    print(f"On page {page_num}")

    url = BASE_URL + PAGE_PARAM + str(page_num)
    response  = requests.get(url)

    if response.status_code != 200:
        print(f"Fehler beim Laden von Seite {page_num}")
        break

    content = response.text
    soup = BeautifulSoup(content, "html.parser")

    teams = soup.find_all("tr", class_="team")

    for team in teams:
        team_name = team.find("td", class_="name").text.strip()
        year = team.find("td", class_="year").text.strip()
        wins = team.find("td", class_="wins").text.strip()
        losses = team.find("td", class_="losses").text.strip()
        goals_for = team.find("td", class_="gf").text.strip()
        goals_against = team.find("td", class_="ga").text.strip()
        diff = team.find("td", class_="diff").text.strip()

        all_teams_data.append([team_name, year, wins, losses, goals_for, goals_against, diff])

df = pd.DataFrame(all_teams_data, columns=["Team", "Jahr", "Siege", "Niederlagen", "GF", "GA", "+/-"])
df.to_csv("nhl_teams_all_pages.csv", index=False)
print(df)
```

Einen Nachteil hat der Code, die Anzahl der Seiten ist statisch `for page_num in range(1, 25)`. Dieses Problem können wir jedoch lösen:
-  Methode 1: Die letzte Seitenzahl aus der Paginierung auslesen
-  Methode 2: Prüfen, ob eine Seite existiert

**Methode 1:**

```python
from bs4 import BeautifulSoup
import requests
import pandas as pd

BASE_URL = "https://www.scrapethissite.com/pages/forms/"
PAGE_PARAM = "?page_num="

all_teams_data = []
page_numbers = []

response = requests.get(BASE_URL)
if response.status_code != 200:
    print("Fehler beim Laden!")

soup = BeautifulSoup(response.text, "html.parser")
pagination_links = soup.select("ul.pagination a")

for link in pagination_links:
    number_str = link.text.strip()
    if number_str.isdigit(): 
        page_numbers.append(int(number_str))

if page_numbers:
    max_page = max(page_numbers)
else:
    max_page = 1

print(f"Die letzte verfügbare Seite ist: {max_page}")

all_teams_data = []

for page_num in range(1, max_page):
    print(f"On page {page_num}")

    url = BASE_URL + PAGE_PARAM + str(page_num)
    response  = requests.get(url)

    if response.status_code != 200:
        print(f"Fehler beim Laden von Seite {page_num}")
        break

    content = response.text
    soup = BeautifulSoup(content, "html.parser")

    teams = soup.find_all("tr", class_="team")

    for team in teams:
        team_name = team.find("td", class_="name").text.strip()
        year = team.find("td", class_="year").text.strip()
        wins = team.find("td", class_="wins").text.strip()
        losses = team.find("td", class_="losses").text.strip()
        goals_for = team.find("td", class_="gf").text.strip()
        goals_against = team.find("td", class_="ga").text.strip()
        diff = team.find("td", class_="diff").text.strip()

        all_teams_data.append([team_name, year, wins, losses, goals_for, goals_against, diff])

df = pd.DataFrame(all_teams_data, columns=["Team", "Jahr", "Siege", "Niederlagen", "GF", "GA", "+/-"])
df.to_csv("nhl_teams_all_pages.csv", index=False)
print(df)
```

**Methode 2:**

Ist leider in dem Fall nicht möglich, da solche Seiten wie z.B `https://www.scrapethissite.com/pages/forms/?page_num=29` vorhanden sind, jedoch ohne Daten.

**Code umstrukturieren:**

- Der Code soll in Funktionen unterteilt sein
- Die Datentypen sollen klar und deutlich erkennbar sein

```python
from bs4 import BeautifulSoup
import requests
import pandas as pd
from typing import Optional

BASE_URL: str = "https://www.scrapethissite.com/pages/forms/"
PAGE_PARAM: str = "?page_num="

def get_last_page() -> int:
    """
    Findet die letzte verfügbare Seite anhand der Paginierung.
    Falls keine Paginierung vorhanden ist oder ein Fehler auftritt, wird die Standardseite 1 zurückgegeben.
    
    Returns:
        int: Die höchste Seitenzahl oder 1, falls keine Paginierung existiert.
    """
    response: requests.Response = requests.get(BASE_URL)
    
    if response.status_code != 200:
        print("Fehler beim Laden der Hauptseite!")
        return 1  

    soup: BeautifulSoup = BeautifulSoup(response.text, "html.parser")
    pagination_links = soup.select("ul.pagination a")

    page_numbers: list[int] = [int(link.text.strip()) for link in pagination_links if link.text.strip().isdigit()]
    
    return max(page_numbers) if page_numbers else 1

def scrape_team_data() -> Optional[pd.DataFrame]:
    """
    Ruft die Daten aller Seiten ab und gibt sie als DataFrame zurück.
    
    Returns:
        Optional[pd.DataFrame]: Enthält die Team-Daten, falls erfolgreich, sonst None.
    """
    last_page: int = get_last_page()
    all_teams_data: list[list[str]] = []

    for page_num in range(1, last_page + 1):
        print(f"Lade Daten von Seite {page_num}...")
        
        url: str = BASE_URL + PAGE_PARAM + str(page_num)
        response: requests.Response = requests.get(url)

        if response.status_code != 200:
            print(f"Fehler beim Laden von Seite {page_num}")
            continue

        soup: BeautifulSoup = BeautifulSoup(response.text, "html.parser")
        teams = soup.find_all("tr", class_="team")

        for team in teams:
            team_name: str = team.find("td", class_="name").text.strip()
            year: str = team.find("td", class_="year").text.strip()
            wins: str = team.find("td", class_="wins").text.strip()
            losses: str = team.find("td", class_="losses").text.strip()
            goals_for: str = team.find("td", class_="gf").text.strip()
            goals_against: str = team.find("td", class_="ga").text.strip()
            diff: str = team.find("td", class_="diff").text.strip()

            all_teams_data.append([team_name, year, wins, losses, goals_for, goals_against, diff])

    if all_teams_data:
        df: pd.DataFrame = pd.DataFrame(all_teams_data, columns=["Team", "Jahr", "Siege", "Niederlagen", "GF", "GA", "+/-"])
        return df
    else:
        print("Keine Daten gefunden!")
        return None

def save_data_to_csv(df: pd.DataFrame, filename: str = "nhl_teams_all_pages.csv") -> None:
    """
    Speichert den DataFrame als CSV-Datei.

    Args:
        df (pd.DataFrame): Der DataFrame, der gespeichert werden soll.
        filename (str): Der Name der Datei (Standard: 'nhl_teams_all_pages.csv').

    Returns:
        None
    """
    if df is not None:
        df.to_csv(filename, index=False)
        print(f"\nDaten erfolgreich gespeichert als: {filename}")
    else:
        print("Keine Daten zum Speichern vorhanden!")

if __name__ == "__main__":
    df: Optional[pd.DataFrame] = scrape_team_data()
    save_data_to_csv(df)
```

**Explorative Datenanalyse:**

Jetzt können wir mit den Daten eine einfache explorative Datenanalyse betreiben und interessante Einblicke bekommen:

```python
from bs4 import BeautifulSoup
import requests
import pandas as pd
from typing import Optional
import matplotlib.pyplot as plt

BASE_URL: str = "https://www.scrapethissite.com/pages/forms/"
PAGE_PARAM: str = "?page_num="

def get_last_page() -> int:
    """
    Findet die letzte verfügbare Seite anhand der Paginierung.
    Falls keine Paginierung vorhanden ist oder ein Fehler auftritt, wird die Standardseite 1 zurückgegeben.
    
    Returns:
        int: Die höchste Seitenzahl oder 1, falls keine Paginierung existiert.
    """
    response: requests.Response = requests.get(BASE_URL)
    
    if response.status_code != 200:
        print("Fehler beim Laden der Hauptseite!")
        return 1  

    soup: BeautifulSoup = BeautifulSoup(response.text, "html.parser")
    pagination_links = soup.select("ul.pagination a")

    page_numbers: list[int] = [int(link.text.strip()) for link in pagination_links if link.text.strip().isdigit()]
    
    return max(page_numbers) if page_numbers else 1

def scrape_team_data() -> Optional[pd.DataFrame]:
    """
    Ruft die Daten aller Seiten ab und gibt sie als DataFrame zurück.
    
    Returns:
        Optional[pd.DataFrame]: Enthält die Team-Daten, falls erfolgreich, sonst None.
    """
    last_page: int = get_last_page()
    all_teams_data: list[list[str]] = []

    for page_num in range(1, last_page + 1):
        print(f"Lade Daten von Seite {page_num}...")
        
        url: str = BASE_URL + PAGE_PARAM + str(page_num)
        response: requests.Response = requests.get(url)

        if response.status_code != 200:
            print(f"Fehler beim Laden von Seite {page_num}")
            continue

        soup: BeautifulSoup = BeautifulSoup(response.text, "html.parser")
        teams = soup.find_all("tr", class_="team")

        for team in teams:
            team_name: str = team.find("td", class_="name").text.strip()
            year: str = team.find("td", class_="year").text.strip()
            wins: str = team.find("td", class_="wins").text.strip()
            losses: str = team.find("td", class_="losses").text.strip()
            goals_for: str = team.find("td", class_="gf").text.strip()
            goals_against: str = team.find("td", class_="ga").text.strip()
            diff: str = team.find("td", class_="diff").text.strip()

            all_teams_data.append([team_name, year, wins, losses, goals_for, goals_against, diff])

    if all_teams_data:
        df: pd.DataFrame = pd.DataFrame(all_teams_data, columns=["Team", "Jahr", "Siege", "Niederlagen", "GF", "GA", "+/-"])
        return df
    else:
        print("Keine Daten gefunden!")
        return None

def save_data_to_csv(df: pd.DataFrame, filename: str = "nhl_teams_all_pages.csv") -> None:
    """
    Speichert den DataFrame als CSV-Datei.

    Args:
        df (pd.DataFrame): Der DataFrame, der gespeichert werden soll.
        filename (str): Der Name der Datei (Standard: 'nhl_teams_all_pages.csv').

    Returns:
        None
    """
    if df is not None:
        df.to_csv(filename, index=False)
        print(f"\nDaten erfolgreich gespeichert als: {filename}")
    else:
        print("Keine Daten zum Speichern vorhanden!")

# Ab hier Datenanalyse

def load_data(file_name: str) -> pd.DataFrame:
    """Lädt die CSV-Datei und gibt einen DataFrame zurück."""
    df = pd.read_csv(file_name)
    return df

def avg_wins_losses_per_year(df: pd.DataFrame) -> None:
    """Zeigt die durchschnittlichen Siege und Niederlagen pro Jahr an."""
    yearly_avg = df.groupby("Jahr")[["Siege", "Niederlagen"]].mean()
    
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(yearly_avg.index, yearly_avg["Siege"], marker='o', label="Durchschnittliche Siege")
    ax.plot(yearly_avg.index, yearly_avg["Niederlagen"], marker='o', label="Durchschnittliche Niederlagen")
    
    ax.set_title("Durchschnittliche Siege und Niederlagen pro Jahr")
    ax.set_xlabel("Jahr")
    ax.set_ylabel("Anzahl")
    ax.legend()
    ax.grid()
    
    plt.show()

if __name__ == "__main__":
    # df: Optional[pd.DataFrame] = scrape_team_data()
    # save_data_to_csv(df)
    data = load_data("nhl_teams_all_pages.csv")
    print(data)
    avg_wins_losses_per_year(data)
```

Ein schönes User-Interface mit streamlit kann das Projekt angenehmer gestallten:

